(selective_sweeps)=

# Selective sweeps

In [1]:
import fwdpy11
import numpy as np
import msprime

In [2]:
import fwdpy11.conditional_models
import fwdpy11.tskit_tools

In [3]:
def setup(prune_selected=False):
    # Dropping mutations requires existing
    # ancestry, which we can get either
    # from a burn-in or from msprime.
    initial_ts = msprime.sim_ancestry(
        samples=500,
        population_size=500,
        recombination_rate=1e-1,
        random_seed=43215,
        sequence_length=1.0,
    )

    # Build the pop from msprime output
    pop = fwdpy11.DiploidPopulation.create_from_tskit(initial_ts)

    # Set up basic model parameters
    pdict = {
        "recregions": [fwdpy11.PoissonInterval(0, 1, 1e-1)],
        # Here, 2 means that fitness is multiplicative
        # over 1, 1+hs, 1+2s.
        "gvalue": fwdpy11.Multiplicative(2.0),
        "rates": (0, 0, None),
        "prune_selected": False,
        "simlen": 200,
    }
    params = fwdpy11.ModelParams(**pdict)

    return pop, params

## From a new mutation

In [4]:
ALPHA = 1000.0

In [5]:
rng = fwdpy11.GSLrng(12345)
pop, params = setup()

In [6]:
mutation_data = fwdpy11.conditional_models.NewMutationParameters(
    frequency=fwdpy11.conditional_models.AlleleCount(1),
    data=fwdpy11.NewMutationData(effect_size=ALPHA / 2 / pop.N, dominance=1),
    position=fwdpy11.conditional_models.PositionRange(left=0.49, right=0.51),
)

In [7]:
output = fwdpy11.conditional_models.selective_sweep(
    rng, 
    pop,
    params,
    mutation_data,
    fwdpy11.conditional_models.GlobalFixation()
)

In [8]:
assert output.pop.generation == params.simlen
assert pop.generation == 0

In [9]:
print(output.pop.mutations[output.index])

Mutation[position:0.507803, effect size:1.000000, dominance:1.000000, origin time:0, label:0]


In [10]:
for fixation, time in zip(output.pop.fixations, output.pop.fixation_times):
    print(fixation, time)

Mutation[position:0.507803, effect size:1.000000, dominance:1.000000, origin time:0, label:0] 26


In [11]:
FIXATION_TIME = output.pop.fixation_times[0]

### Recording the generation when fixation happened

In [12]:
rng = fwdpy11.GSLrng(12345)

In [13]:
output = fwdpy11.conditional_models.selective_sweep(
    rng, 
    pop,
    params,
    mutation_data,
    fwdpy11.conditional_models.GlobalFixation(),
    sampling_policy=fwdpy11.conditional_models.AncientSamplePolicy.COMPLETION,
)

In [14]:
assert len(output.pop.ancient_sample_nodes) == 2 * output.pop.N
assert output.pop.fixation_times[output.index] == FIXATION_TIME

In [15]:
node_array = np.array(output.pop.tables.nodes, copy=False)
ancient_sample_node_times = \
    node_array["time"][output.pop.ancient_sample_nodes]
assert np.all([ancient_sample_node_times == \
    output.pop.fixation_times[output.index]])

## From a standing variant

The recipes for a standing variant are identical to those show above, except that one uses {class}`fwdpy11.conditional_models.AlleleCountRange` or {class}`fwdpy11.conditional_models.FrequencyRange` to specify the starting frequencies.